In [4]:
# 04_agent_with_tools.py
import os
import uuid
import json
import logging
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import HumanMessage

# ✅ NEW (LangChain v1+)
from langchain.agents import create_agent


# ---------- basic structured logging ----------
logger = logging.getLogger("agent_tools")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter("%(asctime)s %(levelname)s %(message)s"))
logger.handlers = [handler]

def jlog(event: str, **fields):
    payload = {"ts": datetime.utcnow().isoformat() + "Z", "event": event, **fields}
    logger.info(json.dumps(payload, ensure_ascii=False))


@tool
def word_count(text: str) -> int:
    """Return the number of words in the given text."""
    return len([w for w in text.strip().split() if w])


def main():
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

    # Tools
    search = DuckDuckGoSearchRun()
    tools = [search, word_count]

    # ✅ Create v1 agent (replaces create_react_agent + AgentExecutor)
    agent = create_agent(
        model=llm,
        tools=tools,
        debug=True,
        system_prompt=(
            "You are a tool-using agent.\n"
            "- Use tools when they help.\n"
            "- If you use search, summarize results and cite sources if shown.\n"
            "- Be accurate and concise.\n"
        ),
    )

    session_id = os.getenv("SESSION_ID", "local-dev")
    print("Type a task. Type exit to quit.")

    while True:
        text = input("\nTask> ").strip()
        if not text or text.lower() in {"exit", "quit"}:
            break

        request_id = str(uuid.uuid4())[:8]
        jlog(
            "request.start",
            session_id=session_id,
            request_id=request_id,
            input=text,
            tools=[t.name if hasattr(t, "name") else type(t).__name__ for t in tools],
            model="gpt-4o-mini",
        )

        try:
            # v1 agents take messages input
            out = agent.invoke(
                {"messages": [HumanMessage(content=text)]},
                config={
                    "tags": ["program-4", "tools", "debug"],
                    "metadata": {"session_id": session_id, "request_id": request_id},
                },
            )

            messages = out.get("messages", [])
            answer = messages[-1].content if messages else str(out)

            jlog(
                "request.end",
                session_id=session_id,
                request_id=request_id,
                output=answer,
            )

            print("\nFinal:\n", answer)
            print("\nTrace:", {"session_id": session_id, "request_id": request_id})

        except Exception as e:
            jlog(
                "request.error",
                session_id=session_id,
                request_id=request_id,
                error=str(e),
            )
            print("\nError:", e)


if __name__ == "__main__":
    main()

Type a task. Type exit to quit.


/var/folders/vk/j_r2f1rs58q7c4bywftwhkgm0000gn/T/ipykernel_91614/2669469455.py:28: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  payload = {"ts": datetime.utcnow().isoformat() + "Z", "event": event, **fields}
2026-02-19 13:25:37,567 INFO {"ts": "2026-02-19T19:25:37.567440Z", "event": "request.start", "session_id": "local-dev", "request_id": "9cc3625a", "input": "Can you tell me the current weather for Minneaspolis?", "tools": ["duckduckgo_search", "word_count"], "model": "gpt-4o-mini"}


[values] {'messages': [HumanMessage(content='Can you tell me the current weather for Minneaspolis?', additional_kwargs={}, response_metadata={}, id='e7b9d346-f90d-4214-b239-5f8f8dc0e0e2')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 144, 'total_tokens': 163, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DB3wYM72FkypKK4noI9UK2tCVf3At', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c775d-6dac-7351-91eb-5a699c52e4cc-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current weather Minneapolis'}, 'id': 'call_rlROWa5USC2lzakeU54

/var/folders/vk/j_r2f1rs58q7c4bywftwhkgm0000gn/T/ipykernel_91614/2669469455.py:28: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  payload = {"ts": datetime.utcnow().isoformat() + "Z", "event": event, **fields}
2026-02-19 13:25:42,368 INFO {"ts": "2026-02-19T19:25:42.368458Z", "event": "request.end", "session_id": "local-dev", "request_id": "9cc3625a", "output": "The current weather in Minneapolis is overcast with a temperature of -3°C (approximately 27°F). The humidity is at 89%, and there is a wind speed of 10.4 km/h. The weather feels like -7°C (approximately 19°F). A winter weather advisory is in effect until Thursday at 8:00 AM CST."}


[updates] {'model': {'messages': [AIMessage(content='The current weather in Minneapolis is overcast with a temperature of -3°C (approximately 27°F). The humidity is at 89%, and there is a wind speed of 10.4 km/h. The weather feels like -7°C (approximately 19°F). A winter weather advisory is in effect until Thursday at 8:00 AM CST.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 444, 'total_tokens': 517, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_414ba99a04', 'id': 'chatcmpl-DB3wbKGLfQWPBT9pm6v3MHntN7JzZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c775d-7750-7bd1-b0b5-202339e2f201-0', tool_calls=[], invalid_tool_calls=[], us